In [ ]:
## Install a few packages that aren't included in our Docker image

!pip install --upgrade pip
!apt-get -y install libxml2-dev libxslt-dev 
!pip3 install newspaper3k

In [ ]:
!mkdir /sharedfolder/nyt_articles_2017-04-12/

import os

os.chdir('/sharedfolder/nyt_articles_2017-04-12/')

In [ ]:
import os
import time
from urllib.request import urlopen
from newspaper import Article
from bs4 import BeautifulSoup

## Creating a function that uses the 'newspaper' package to download an article and return its text

def url_to_article_text(url):
    article = Article(url)
    article.download()
    article.parse()
    article_text = article.text.replace('\n', ' ')
    return article_text

In [ ]:
## Creating a list of NYT section pages

section_urls = [ \
'https://www.nytimes.com/section/health', \
'https://www.nytimes.com/section/arts', \
'https://www.nytimes.com/section/fashion', \
'https://www.nytimes.com/section/world', \
'https://www.nytimes.com/section/us', \
'https://www.nytimes.com/section/politics', \
'https://www.nytimes.com/section/nyregion', \
'https://www.nytimes.com/section/business', \
'https://www.nytimes.com/section/opinion', \
'https://www.nytimes.com/section/technology', \
'https://www.nytimes.com/section/science', \
'https://www.nytimes.com/section/health', \
'https://www.nytimes.com/section/sports', \
'https://www.nytimes.com/section/food', \
'https://www.nytimes.com/section/travel', \
'https://www.nytimes.com/section/t-magazine', \
'https://www.nytimes.com/section/magazine', \
'https://www.nytimes.com/section/realestate']

In [ ]:
## Shuffling the order of NYT pages

import random

random.shuffle(section_urls)

In [ ]:
## Completing the scrape
# We're downloading each section's articles to its own directory.

for section_url in section_urls:
    print(section_url)
    
    section_name = section_url.split('/')[-1]              ## Excerpting the current section name from the URL
    
    try: os.mkdir('/sharedfolder/nyt_articles_2017-04-12/' + section_name)   ## Creating the section directory, 
    except: pass                                                             # if it doesn't already exist
    
    os.chdir('/sharedfolder/nyt_articles_2017-04-12/' + section_name)        ## Changing to that directory
    
    html_text = urlopen(section_url)
    soup = BeautifulSoup(html_text, 'lxml')                                            ## Parsing the page with BeautifulSoup
    
    urls = [item['href'] for item in soup.find_all('a') if '.html' in item['href']]    ## Extracting a list of link URLs

    try: urls = random.sample(urls, 25)                                         ## Picking 25 article URLs at random
    except: pass
    
    for url in urls:
        try:
            article_text = url_to_article_text(url)                 ## Using the article downloading function we created above  
            filename = url.split('/')[-1].split('.html')[0]+'.txt'              ## Creating our new filename
            if filename not in os.listdir('./'):                                ## Writing .txt file
                with open(filename, 'w') as fo:
                    fo.write(article_text)
            time.sleep(0.5 + random.random())                                   ## Pausing before downloading next page
        except Exception as e:
            print('----')
            print(url)
            print(e)
            print('----')
    
    time.sleep(3 + random.random())                                             ## Pausing before downloading next section

In [ ]:
## Grouping all articles in the same folder

os.chdir('/sharedfolder/nyt_articles_2017-04-12/')

!mkdir all_articles

!mv ./*/*.txt ./all_articles/

In [ ]:
## Zipping up the whole collection

os.chdir('/sharedfolder/')

!zip -r nyt_articles_2017-04-12.zip nyt_articles_2017-04-12/